In [27]:
import requests
from typing import Optional, List, Dict, Any
from langchain.llms.base import LLM

class LocalLLM(LLM):
    # ประกาศฟิลด์ให้ Pydantic รู้จัก
    api_url: str
    model_name: str

    @property
    def _llm_type(self) -> str:
        """ชื่อชนิดของ LLM เพื่อ logging ภายใน LangChain"""
        return "local_llm"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model": self.model_name, "api_url": self.api_url}

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> str:
        """เรียก API และคืน response มาเป็นสตริง"""
        res = requests.post(
            f"{self.api_url}/api/generate",
            json={
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,
            },
        )
        res.raise_for_status()
        return res.json().get("response", "")

API_URL = "http://localhost:5433"
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"
llm = LocalLLM(api_url=API_URL, model_name=MODEL_NAME)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
# Wrap SentenceTransformer ผ่าน LangChain
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

C:\Users\Arther\AppData\Local\Temp\ipykernel_21040\2569073825.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
d:\TTT_Trainee\Project_1\AI-Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\TTT_Trainee\Project_1\AI-Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from

In [72]:
from langchain.document_loaders import TextLoader
import glob

paths = glob.glob("Raw-data-from-TTT\**\*.txt", recursive=True)
documents = []
for p in paths:
    loader = TextLoader(p, encoding="utf-8")
    documents.extend(loader.load())


In [73]:
from langchain.vectorstores import FAISS
# สร้างและเก็บ Document embeddings ลงใน FAISS index
vectorstore = FAISS.from_documents(documents, embeddings)
# บันทึก index ไว้ใช้งานครั้งถัดไป
vectorstore.save_local("faiss_index")

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Custom prompt template
custom_template = """ใช้บริบทต่อไปนี้ในการตอบคำถามที่อยู่ท้ายบท
ผมคือ TTT-Assistant ผู้ช่วย AI ของบริษัท TTT Brothers Co., Ltd.
หากคุณไม่ทราบคำตอบ ให้ตอบเพียงว่าคุณไม่ทราบ อย่าพยายามแต่งคำตอบขึ้นมา

ข้อมูลที่มี:
{context}

คำถาม: {question}

คำตอบ: """

CUSTOM_PROMPT = PromptTemplate(
    template=custom_template,
    input_variables=["context", "question"]
)

# สร้าง Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# สร้าง QA Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT}
)

In [ ]:
# คำถามตัวอย่าง
question = "แปลว่าอะไร"
# เรียก chain พร้อมส่งคำถาม
result = qa_chain({"question": question})

# ดูประวัติการสนทนา
#print("Chat history:\n", memory.load_memory_variables({}))

print(result.get("answer"))

จากข้อมูลที่ให้มา วิสัยทัศน์ของบริษัท TTT Brothers Co., Ltd. คือ "how the world works simplifying" หมายถึง การทำให้ทุกอย่างง่ายขึ้น
